<a href="https://colab.research.google.com/github/komazawa-deep-learning/komazawa-deep-learning.github.io/blob/master/2021notebooks/2021_1008face_dataset_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 顔データベースによる機械学習のデモと PyTorch による回帰，正則化項の実習

- author: 浅川伸一
- date: 2021_1008
- filename: 2021_1008face_dataset_regression.ipynb
- license: MIT



In [ ]:
import numpy as np
from sklearn.datasets import fetch_olivetti_faces
import matplotlib.pyplot as plt
!pip install japanize_matplotlib
import japanize_matplotlib
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

data = fetch_olivetti_faces()
X, y = data.data, data.target

In [ ]:
# n 番目の画像を表示してみましょう
n = int(input('0 から 399 までの数字を一つ入力してください ')) 
plt.imshow(X[n].reshape(64,64), cmap='gray')

In [ ]:
# fig1, fig1_axes = plt.subplots(ncols=10, nrows=3, figsize=(20,6)) # , constrained_layout=True)
# for i in range(3):
#     for j in range(10):
#         x = i * 10 + j
#         fig1_axes[i][j].imshow(X[x].reshape(64,64), cmap='gray')
#         fig1_axes[i][j].axis('off')        

In [ ]:
target = y
print("目標とするクラス(画像中の人物の数) :", np.unique(target))

# 1. 機械学習手法による顔認識

## 1.1 データの分割，訓練データとテストデータ

オリベッティ顔データセットには， 各被験者の 10 枚の顔画像が含まれています。
このうち 80% を訓練データとし，20% をテストデータとして使用します。
各被験者の訓練画像とテスト画像の数が同じになるように stratify 機能を使用します。
したがって，各被験者には 8 枚の訓練用画像と 2 枚のテスト用画像が用意されることになります。
訓練データとテストデータの割合は変更することができます。

In [ ]:
#import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import metrics
import seaborn as sns


# test_size = 0.2 としているので，訓練データ対テストデータが 8:2 になります
X_train, X_test, y_train, y_test=train_test_split(X, target, test_size=0.2, stratify=target, random_state=0)
print(f'X_train 訓練画像のサイズ: {X_train.shape}')
print(f'y_train 教師信号データのサイズ: {y_train.shape}')

## 分類手法の検討

訓練画像から人物が予測できるか否かを検討します。

In [ ]:
clf = SVC()  # サポートベクターマシンを宣言
clf.fit(X_train, y_train)     # 訓練データを用いてサポートベクターマシンモデルを訓練
y_hat = clf.predict(X_test)  # テストデータを使って予測を行い結果を y_hat に格納
print(f"サポートベクターマシンを用いた分類精度: {metrics.accuracy_score(y_test, y_hat):.3f}")

# 混同行列の表示
plt.figure(figsize=(10,8))
sns.heatmap(metrics.confusion_matrix(y_test, y_hat))

In [ ]:
# 線形判別分析による予測
clf = LinearDiscriminantAnalysis()
clf.fit(X_train, y_train)    # 訓練データを用いて線形判別分析モデルを訓練
y_hat = clf.predict(X_test)  # テストデータを使って予測を行い結果を y_hat に格納
print(f"線形判別分析を用いた分類精度: {metrics.accuracy_score(y_test, y_hat):.3f}")

# 混同行列の表示
plt.figure(1, figsize=(10,8))
sns.heatmap(metrics.confusion_matrix(y_test, y_hat))

In [ ]:
# ロジスティック回帰による予測
log_reg = LogisticRegression(max_iter=10 ** 4)
log_reg.fit(X_train, y_train)    # 訓練データを用いて線形判別分析モデルを訓練
y_hat = log_reg.predict(X_test)  # テストデータを使って予測を行い結果を y_hat に格納
print(f"ロジスティック回帰を用いた分類精度: {metrics.accuracy_score(y_test, y_hat):.3f}")

# 混同行列の表示
plt.figure(figsize=(10,8))
sns.heatmap(metrics.confusion_matrix(y_test, y_hat))

# 交差検証

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

for name, model in [['サポートベクターマシン', SVC()], 
                    ['線形判別分析', LinearDiscriminantAnalysis()],
                    ['ロジスティック回帰', LogisticRegression(max_iter= 10 ** 4)]]:
    kfold = KFold(n_splits=5, shuffle=True, random_state=0)
    cv_scores = cross_val_score(model, X_train, y_train, cv=kfold)
    print(f"{name} 平均交差検証得点: {cv_scores.mean():.2f}")

## より多くの検証結果を得るための リーブ・ワン・アウト 交差検証

オリベッティ顔データセットには，各被験者に対して 10 枚の顔画像が含まれています。
これは， 機械学習モデルの学習やテストには少ない数です。

クラスの例が少ない機械学習モデルをよりよく評価するために， 採用される交差検証法にリーブ・ワン・アウト leave-one-out (LOO) 交差検証法があります。
LOO 法では，あるクラスのサンプルのうち 1 つだけをテストに使用します。
他のサンプルは訓練に使用します。
この手順を， 全サンプルを一度づつテストに使用して繰り返さします。


In [ ]:
from sklearn.model_selection import LeaveOneOut

loo_cv = LeaveOneOut()
clf = LinearDiscriminantAnalysis()
cv_scores = cross_val_score(clf,
                            X_train,
                            y_train,
                            cv = loo_cv)

print(f"{clf.__class__.__name__} リーブ・ワン・アウト交差検証法による平均得点:{cv_scores.mean():.3f}")

In [ ]:
#help(clf.__class__)
#dir(clf)  # .__class__) #.coef_)
#dir(clf.__class__) # n_components) #_get_param_names())
# coef_ 
# intercept_
# covariance_ 
cv_scores

## ハイパーパラメータの調整。GridSearcCV

モデルの汎化性能向上のために GridSearchCV を行います。
ロジスティック回帰分類器のハイパーパラメータを調整してみます

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import LeaveOneOut

params={'penalty':['l1', 'l2'],
        'C':np.logspace(0, 4, 10) }
clf = LogisticRegression()
loo_cv = LeaveOneOut()
gridSearchCV = GridSearchCV(clf, params, cv=loo_cv)
gridSearchCV.fit(X_train, y_train)
print("Grid search fitted..")
print(gridSearchCV.best_params_)
print(gridSearchCV.best_score_)
print(f"グリッドサーチによる交差妥当性得点:{gridSearchCV.score(X_test, y_test):.3f}")

## 精度ー再現率 - ROC曲線

精度ー再現率曲線は 2 値分類の場合です。
オリベッティ顔データセットでは 40 の異なるクラス (40人分の顔画像) があります。

In [ ]:
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier

Target = label_binarize(target, classes=range(40))
n_classes = Target.shape[1]
X_train_multiclass, X_test_multiclass, \
y_train_multiclass, y_test_multiclass = train_test_split(X, 
                                                         Target,
                                                         test_size=0.2,
                                                         stratify=Target,
                                                         random_state=0)

oneRestClassifier = OneVsRestClassifier(clf)
oneRestClassifier.fit(X_train_multiclass, y_train_multiclass)
y_score = oneRestClassifier.decision_function(X_test_multiclass)

precision = dict()
recall = dict()
average_precision = dict()
for i in range(n_classes):
    precision[i], recall[i], _ = metrics.precision_recall_curve(y_test_multiclass[:, i], y_score[:, i])
    average_precision[i] = metrics.average_precision_score(y_test_multiclass[:, i], y_score[:, i])

precision["micro"], recall["micro"], _ = metrics.precision_recall_curve(y_test_multiclass.ravel(), y_score.ravel())
average_precision["micro"] = metrics.average_precision_score(y_test_multiclass, y_score, average="micro")
print(f'平均精度得点, 全クラスの平均: {average_precision["micro"]:0.3f}')

In [ ]:
!pip install install funcsigs
from funcsigs import signature 

step_kwargs = ({'step': 'post'}
               if 'step' in signature(plt.fill_between).parameters
               else {})
plt.figure(1, figsize=(12,8))
plt.step(recall['micro'], precision['micro'], color='b', alpha=0.2,
         where='post')
plt.fill_between(recall["micro"], precision["micro"], alpha=0.2, color='b',
                 **step_kwargs)

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title(
    'Average precision score, micro-averaged over all classes: AP={0:0.2f}'
    .format(average_precision["micro"]))

# ここからは，ニューラルネットワーク

In [ ]:
print(f'y_train のサイズ: {y_train.shape}')
print(f'X_train のサイズ: {X_train.shape}')
n_classes = len(set(y_train))
print(f'n_classes: {n_classes}')

In [ ]:
import torch
import torch.nn as nn

X_ = torch.tensor(X_train).float()
y_ = torch.tensor(y_train).long()
print(f'X_ のサイズ: {X_.size()}')
print(f'y_ のサイズ: {y_.size()}')

# 線形回帰モデルを PyTorch で定義
class lin_reg_Module(nn.Module):
    def __init__(self, n_input, n_output):
        super().__init__()
        self.fc = nn.Linear(n_input, n_output, bias=True)

    def forward(self, x):
        y_hat = self.fc(x)
        return y_hat

# LinearModelのインスタンス作成
reg_model = lin_reg_Module(n_input = X_.size(1),
                           n_output = n_classes)

# 回帰モデルの状態辞書を表示
print(reg_model.state_dict())

In [ ]:
import torch 
from torch import nn 

#loss_f = nn.MSELoss()         # 損失関数 平均自乗誤差
loss_f = nn.CrossEntropyLoss() # 損失関数 交差エントロピー
optimizer = torch.optim.Adam(params = reg_model.parameters(), lr=0.01)

In [ ]:
n_epochs = 50
loss_list = [] #損失関数の値を保存するためのリスト

reg_model0 = lin_reg_Module(n_input = X_.size(1),
                            n_output = n_classes)
reg_model0.fc.reset_parameters()
loss_f = nn.CrossEntropyLoss() # 損失関数 交差エントロピー
optimizer = torch.optim.Adam(params = reg_model0.parameters(), lr=0.01)

for epoch in range(n_epochs):
    y_hat = reg_model0(X_)    # 予測値の計算
    loss = loss_f(y_hat, y_)  # 損失関数の値を計算
    optimizer.zero_grad()     # 勾配を初期化
    loss.backward()           # 勾配を計算
    optimizer.step()          # パラメータを更新
    loss_list.append(loss.detach()) # 損失関数の値を保存


In [ ]:
import matplotlib.pyplot as plt

print(loss_list[-1])
plt.plot(loss_list)
plt.xlabel('反復訓練回数') 
plt.ylabel('損失値') 
plt.title('正則化項なし')
plt.show()

In [ ]:
print(reg_model.fc.weight)
print(reg_model.fc.bias)

In [ ]:
# パラメータ初期化
reg_model.fc.reset_parameters()
print(reg_model.fc.weight)
print(reg_model.fc.bias)

In [ ]:
n_epochs = 50
loss_list = []
Lambda = 1 # 正則化パラメータ

reg_model1 = lin_reg_Module(n_input = X_.size(1),
                            n_output = n_classes)
reg_model1.fc.reset_parameters()
loss_f = nn.CrossEntropyLoss() # 損失関数 交差エントロピー
optimizer = torch.optim.Adam(params = reg_model1.parameters(), lr=0.01)

for epoch in range(n_epochs):
    y_hat = reg_model(X_)
    loss = loss_f(y_hat, y_)

    # パラメータのL1ノルムを損失関数に足す
    l1 = torch.tensor(0., requires_grad=True)
    for w in reg_model1.parameters():
        l1 = l1 + torch.norm(w, 1)
        loss = loss + Lambda * l1

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    loss_list.append(loss.detach())

In [ ]:
print(loss_list[-1])
plt.plot(loss_list)
plt.xlabel('反復訓練回数') 
plt.ylabel('損失値') 
plt.show()

In [ ]:
epochs = 50
loss_list = []
Lambda = 0.01 # 正則化パラメータ

reg_model2 = lin_reg_Module(n_input = X_.size(1),
                            n_output = n_classes)
reg_model2.fc.reset_parameters()
loss_f = nn.CrossEntropyLoss() # 損失関数 交差エントロピー
optimizer = torch.optim.Adam(params = reg_model2.parameters(), lr=0.001)

for epoch in range(epochs):
    y_hat = reg_model2(X_)
    loss = loss_f(y_hat, y_)

    # L2 ノルムの二乗を損失関数に加える
    l2 = torch.tensor(0., requires_grad=True)
    for w in reg_model2.parameters():
        l2 = l2 + torch.norm(w) ** 2
        loss = loss + Lambda * l2

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    loss_list.append(loss.detach())

In [ ]:
print(loss_list[-1])
plt.plot(loss_list)
plt.xlabel('反復訓練回数') 
plt.ylabel('損失値') 
plt.show()